<a href="https://colab.research.google.com/github/MykhailoFokin/MachineLearning/blob/Dev/simple_classification_with_MSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import cv2
import fnmatch
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import explained_variance_score

In [3]:
#Copy images from repo to colab
!git clone https://github.com/SilvesterHsu/ORLFaceRecognition-PCA.git

Cloning into 'ORLFaceRecognition-PCA'...
remote: Enumerating objects: 604, done.
remote: Total 604 (delta 0), reused 0 (delta 0), pack-reused 604
Receiving objects: 100% (604/604), 3.59 MiB | 5.58 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [13]:
test_matrix = []
train_matrix = []

test_labels = []
train_labels = []

# read folder structure with files and put it to collection
folder = []
for i in os.walk(os.path.join('ORLFaceRecognition-PCA','att_faces')):
  folder.append(i)

for address, dirs, files in folder:
    
    for filename in fnmatch.filter(files, '*.pgm'):

      img = cv2.imread(os.path.join(address,filename))
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    
      gray.flatten().reshape(1, 10304)
      person = address.replace('ORLFaceRecognition-PCA/att_faces/','')

      # r'[0|8-9]*'
      if fnmatch.filter(filename, r'[0|9]*'):
        test_matrix.append(gray.flatten().reshape(1, 10304))
        test_labels.append(person)
      else:
      #if fnmatch.filter(filename, r'[1-5]*'):
        train_matrix.append(gray.flatten().reshape(1, 10304))
        train_labels.append(person)

np_test_matrix = np.array(test_matrix)
np_train_matrix = np.array(train_matrix)
  
# Different deviation results
result_mse = np.empty((len(np_train_matrix),1))     # mean_squared_error
result_mae = np.empty((len(np_train_matrix),1))     # mean_absolute_error
result_msle = np.empty((len(np_train_matrix),1))    # mean_squared_log_error
result_evs = np.empty((len(np_train_matrix),1))     # explained_variance_score
result_std = np.empty((len(np_train_matrix),1))     # standart deviance

predictions = np.empty((len(np_test_matrix),6),dtype="S3") 

estimations_mse = np.empty((len(np_test_matrix),1)) 
estimations_mae = np.empty((len(np_test_matrix),1)) 
estimations_msle = np.empty((len(np_test_matrix),1)) 
estimations_evs = np.empty((len(np_test_matrix),1)) 
estimations_std = np.empty((len(np_test_matrix),1))

for i in range(len(np_test_matrix)):
  
  for j in range(len(np_train_matrix)):
    result_mse[j] = mean_squared_error(np_test_matrix[i], np_train_matrix[j])
    result_mae[j] = mean_absolute_error(np_test_matrix[i], np_train_matrix[j])
    result_msle[j] = mean_squared_log_error(np_test_matrix[i], np_train_matrix[j])
    result_evs[j] = explained_variance_score(np_test_matrix[i], np_train_matrix[j])
    result_std[j] = np.sqrt(mean_squared_error(np_test_matrix[i], np_train_matrix[j]))
  
  # write source labes with predicted ones
  # comparing predicted and source label in estimation array
  #print(train_labels[np.argmin(result_mse)])
  predictions[i][0] = test_labels[i]
  predictions[i][1] = train_labels[np.argmin(result_mse)]
  predictions[i][2] = train_labels[np.argmin(result_mae)]
  predictions[i][3] = train_labels[np.argmin(result_msle)]
  predictions[i][4] = train_labels[np.argmin(result_evs)]
  predictions[i][5] = train_labels[np.argmin(result_std)]

  estimations_mse[i] = int(test_labels[i]==train_labels[np.argmin(result_mse)])
  estimations_mae[i] = int(test_labels[i]==train_labels[np.argmin(result_mae)])
  estimations_msle[i] = int(test_labels[i]==train_labels[np.argmin(result_msle)])
  estimations_evs[i] = int(test_labels[i]==train_labels[np.argmin(result_evs)])
  estimations_std[i] = int(test_labels[i]==train_labels[np.argmin(result_std)])

#print(predictions)
estimation_result_mse = estimations_mse.sum()/len(estimations_mse)
estimation_result_mae = estimations_mae.sum()/len(estimations_mae)
estimation_result_msle = estimations_msle.sum()/len(estimations_msle)
estimation_result_evs = estimations_evs.sum()/len(estimations_evs)
estimation_result_std = estimations_std.sum()/len(estimations_std)

#print("Estimation for MSE: {} ; MAE: {} ; MSLE: {} ; EVS: {} ; STD: {} ;"
#.format(estimation_result_mse, estimation_result_mae, estimation_result_msle, estimation_result_evs, estimation_result_std))

#print("MSE: {}", estimation_result_mse)
#print("MAE: {}", estimation_result_mae)
#print("MSLE: {}", estimation_result_msle)
#print("EVS: {}", estimation_result_evs)
#print("STD: {}", estimation_result_std)

SCORES = dict(mean_squared_error       = estimation_result_mse,
              mean_absolute_error      = estimation_result_mae,
              mean_squared_log_error   = estimation_result_msle,
              explained_variance_score = estimation_result_evs,
              standart_square_deviance = estimation_result_std)

print(SCORES)

#print(estimations_mse.sum()/len(estimations_mse))

MSE: {} 1.0
MAE: {} 0.075
MSLE: {} 0.925
EVS: {} 0.025
STD: {} 1.0
{'mean_squared_error': 1.0, 'mean_absolute_error': 0.075, 'mean_squared_log_error': 0.925, 'explained_variance_score': 0.025, 'standart_square_deviance': 1.0}
